In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import json
import sys
from pathlib import Path

import ipywidgets as w
import ipyvuetify as v
import pdblp
import numpy as np
import pandas as pd
from ipydatagrid import DataGrid

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
import utils

In [3]:
root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

In [12]:
with open(data_path / 'pairs.json') as f:
    pairs = json.load(f)

In [4]:
indices = pd.read_csv(data_path / 'indices.csv')
indices_history = pd.read_csv(data_path / 'indices_history.csv', parse_dates=['date'], dayfirst=True)

In [5]:
indices = indices.assign(
    combined_name=indices.drop('ticker', axis=1).apply(lambda x: '_'.join(x.dropna()), axis=1),
)

In [6]:
dg = DataGrid(
    dataframe=indices, 
    selection_mode='row',
    base_column_size=90,
    column_widths={
        'ticker': 120,
        'combined_name': 300
    }
)

In [13]:
box = w.VBox(
    children=[
        dg,
        out
    ]
)


def get_tickers():
    c = indices.columns.get_loc('ticker')
    return [indices.iloc[dict_['r'], c] for dict_ in dg.selected_cells if dict_['c'] == c]


def fn(change):
    if len(dg.selections) == 2:
        fig = make_fig(get_tickers())
        with out:
            out.clear_output()
            display(fig)
            
            
dg.observe(fn)

NameError: name 'out' is not defined

In [207]:
def make_fig(tickers):
    
    df = indices_history.loc[indices_history['security'].isin(tickers)] \
                                                        .pivot(index='date', columns='security') \
                                                        .droplevel(level=0, axis=1).dropna()
    ratio = (df.iloc[:, 0] / df.iloc[:, 1]).rename(' / '.join(tickers))
    fig = ratio.plot(backend='plotly',  height=500, template='plotly_white')
    return fig.update_layout(
        legend=
        dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0
        )
    )

In [208]:
idx1_sel = v.Autocomplete(
    v_model=None,
    items=indices['combined_name'].to_list(),
    label='idx1',
    dense=True,
    outlined=True
)

idx2_sel = v.Autocomplete(
    v_model=None,
    items=indices['combined_name'].to_list(),
    label='idx2',
    dense=True,
    outlined=True
)

chart_btn = v.Btn(
    left=True,
    children=[
        v.Icon(children=['mdi-chart-line']),
        'Chart'
    ],
)
out = w.Output()

In [217]:
param_box = v.Container(
    children=[
        v.Row(
            children=[
                v.Col(
                    cols=4,
                    children=[idx1_sel]
                ),
                v.Col(
                    cols=4,
                    children=[idx2_sel]
                ),
                v.Col(
                    cols=2,
                    children=[chart_btn]
                ),
            ]
        )
    ]    
)

In [218]:
tab = w.VBox(
    children=[
        param_box,
        out,
    ]
)

In [219]:
def on_click_chart(widget, event, data):
    chart_btn.loading = True
    try:
        names = [idx1_sel.v_model, idx2_sel.v_model]
        tickers = indices.loc[indices['combined_name'].isin(names), 'ticker'].to_list()
        fig = make_fig(tickers)
        with out:
            out.clear_output()
            display(fig)
    except Exception:
        pass
    finally:
        chart_btn.loading = False

chart_btn.on_event('click', on_click_chart)

### Indices Ratio

In [220]:
tab

In [ ]:
area_sel =  v.Autocomplete(
    v_model=None,
    items=indices['combined_name'].to_list(),
    label='idx2',
    dense=True,
    outlined=True
)
